In [1]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go

In [2]:
bp = pd.read_csv("../preprocess/smooth_bp_eicu2.csv", low_memory=False)

In [61]:
np.random.seed(37)
pat = np.random.choice(bp.stay_id.unique(), 1)
pat = [2893284]

In [ ]:
# plot cur_bp for cur_bp_time for pat
pat_bp = bp[bp.stay_id == pat[0]]
fig = go.Figure()
fig.add_trace(go.Scatter(x=pat_bp.cur_bp_time, y=pat_bp.cur_bp, mode='lines', name='cur_bp'))
fig.add_trace(go.Scatter(x=pat_bp.cur_bp_time, y=pat_bp.otj_filter, mode='lines', name='filters'))
# add a label called "probabily artifact" in the plot
fig.add_annotation(x=pat_bp.cur_bp_time[pat_bp.otj_filter == 1].mean(), y=pat_bp.cur_bp[pat_bp.otj_filter == 1].mean(),
                   text="probabily artifact", showarrow=True, arrowhead=1)
fig.update_layout(title=f"patient {pat[0]}", xaxis_title="time", yaxis_title="cur_bp")
fig.show()


In [193]:
# plot cur_bp for cur_bp_time for pat
pat_bp = bp[bp.stay_id == pat[0]]
fig = go.Figure()
fig.add_trace(go.Scatter(x=pat_bp.cur_bp_time, y=pat_bp.cur_bp, mode='lines', name='cur_bp'))
# fig.add_trace(go.Scatter(x=pat_bp.cur_bp_time, y=pat_bp.otj_filter, mode='lines', name='filters'))
# from (1250, 120) send  arrow to  each point in [(2496, 126), (2301, 93), (2991, 80), (1336, 78), (1896, 100)]
point = (1250, 120)
for i in [(2496, 126), (2301, 93), (2991, 80), (1336, 78), (1896, 100)]:
    fig.add_annotation(x=i[0], y=i[1], xref="x", yref="y",
                       text="probabily artifact", showarrow=True, arrowhead=1)

fig.update_layout(xaxis_title="minutes", yaxis_title="MAP")
fig.write_image("fig1.png")
fig.add_trace(go.Scatter(x=pat_bp.cur_bp_time, y=pat_bp.otj_filter, mode='lines', name='filtered'))
fig.write_image("fig3.png")

fig.show()

In [51]:
# sort patient by #((cur_bp is not null) and (otj_filter is null))
sorted_pats = bp.groupby('stay_id').apply(
    lambda x: x[(x.cur_bp.notnull()) & (x.otj_filter.isnull())].shape[0]).sort_values(ascending=False)


In [107]:
# make a nx2 matrix with cur_bp[i]-cur_bp[i-1] and cur_bp[i+1]-cur_bp[i] for each entry in pat_bp
pat_bp = bp[bp.stay_id == pat[0]]
x = np.c_[pat_bp.cur_bp.diff(), pat_bp.cur_bp.diff(-1)]

In [108]:
# plot the matrix
fig = go.Figure()
fig.add_trace(go.Scatter(x=x[:, 0], y=x[:, 1], mode='markers', name='cur_bp'))
fig.update_layout(title=f"patient {pat[0]}", xaxis_title="cur_bp[i]", yaxis_title="cur_bp[i+1]")
# make plot square
fig.update_layout(
    autosize=False,
    width=500,
    height=500,
    margin=dict(l=50, r=50, b=100, t=100, pad=4)
)
fig.show()


In [110]:
 jumps = np.r_[pat_bp['otj_filter'][1:].values - pat_bp['otj_filter'][:-1].values, [np.nan]]
bi_jump = np.c_[jumps[:-1], jumps[1:]]
bi_jump = np.r_[[[np.nan, np.nan]], bi_jump]

In [192]:
# plot bi_jump
fig = go.Figure()
fig.add_trace(go.Scatter(x=bi_jump[:, 0], y=bi_jump[:, 1], mode='markers', name='changes'))
fig.update_layout(title=f"patient {pat[0]}", xaxis_title="change from previous measurement", yaxis_title="change to next measurement")
# make plot square
fig.update_layout(
    autosize=False,
    width=500,
    height=500,
    margin=dict(l=50, r=50, b=100, t=100, pad=4)
)
# add circle with radius 15 around 0,0
radius = 15.5

angles = np.linspace(0, 2 * np.pi, 100)

# Calculate x and y coordinates for points on the circle
x_coordinates = radius * np.cos(angles)
y_coordinates = radius * np.sin(angles)
fig.add_trace(go.Scatter(x=x_coordinates, y=y_coordinates, mode='lines', name='gradual change', fill='toself'))
# add a x=-y line across the plot and add
fig.add_trace(go.Scatter(x=[np.nanmin(bi_jump[:, 0]), np.nanmax(bi_jump[:, 0])],
                         y=[-np.nanmin(bi_jump[:, 0]), -np.nanmax(bi_jump[:, 0])],
                         mode='lines', name='x=-y'))

distance = np.sqrt(14 ** 2 / 2)
x0 = np.nanmin(bi_jump[:, 0]) - distance
y0 = -np.nanmin(bi_jump[:, 0]) - distance
x1 = np.nanmin(bi_jump[:, 0]) + distance
y1 = -np.nanmin(bi_jump[:, 0]) + distance
x2 = np.nanmax(bi_jump[:, 0]) + distance
y2 = -np.nanmax(bi_jump[:, 0]) + distance
x3 = np.nanmax(bi_jump[:, 0]) - distance
y3 = -np.nanmax(bi_jump[:, 0]) - distance
path = f'M {x0} {y0} L {x1} {y1} L {x2} {y2} L {x3} {y3} Z'
fig.add_trace(
    go.Scatter(x=[x0, x1, x2, x3, x0], y=[y0, y1, y2, y3, y0], mode='lines', name='correction', fill='toself',
               fillcolor='grey', opacity=0.5))

# make axis titles to
fig.write_image("fig2.png")
fig.show()

In [140]:
# compute the distance of each point from the line x=-y



[nan, nan]